## Analyze whether SNWD varies more from year to year or from place to place.

In [1]:
import pandas as pd
import numpy as np
import urllib
import math

In [2]:
import findspark
findspark.init()

from pyspark import SparkContext
#sc.stop()
sc = SparkContext(master="local[3]",pyFiles=['lib/numpy_pack.py','lib/spark_PCA.py','lib/computeStats.py'])

from pyspark import SparkContext
from pyspark.sql import *
sqlContext = SQLContext(sc)

In [3]:
import sys
sys.path.append('./lib')

import numpy as np
from numpy_pack import packArray,unpackArray
from spark_PCA import computeCov
from computeStats import computeOverAllDist, STAT_Descriptions

In [4]:
### Read the data frame from pickle file

data_dir='../../Data/Weather'
file_index='SBBBBBSB'
meas='SNWD'

from pickle import load

#read statistics
filename=data_dir+'/STAT_%s.pickle'%file_index
STAT,STAT_Descriptions = load(open(filename,'rb'))
print('keys from STAT=',STAT.keys())

#!ls -ld $data_dir/*.parquet

#read data
filename=data_dir+'/decon_%s_%s.parquet'%(file_index,meas)

df=sqlContext.read.parquet(filename)
print(df.count())
df.show(2)

('keys from STAT=', ['TMIN', 'TOBS', 'TMAX', 'SNOW', 'SNWD', 'PRCP'])
798
+------------------+------------------+------------------+---------+--------+--------+---------+-----------+-------------------+-------------------+-------------------+-------------------+-----------+-----------+------+--------------------+------+
|           coeff_1|           coeff_2|           coeff_3|elevation|   label|latitude|longitude|measurement|              res_1|              res_2|              res_3|           res_mean|    station|  total_var|undefs|              vector|  year|
+------------------+------------------+------------------+---------+--------+--------+---------+-----------+-------------------+-------------------+-------------------+-------------------+-----------+-----------+------+--------------------+------+
|1811.1595024675585| 460.5274358673977|343.37546092363647|    493.5|SBBBBBSB| 48.8217|-100.4525|       SNWD|0.21585999994825414|0.16516195320429514|0.13697690891565667|0.322776483888

In [5]:
print df.columns

['coeff_1', 'coeff_2', 'coeff_3', 'elevation', 'label', 'latitude', 'longitude', 'measurement', 'res_1', 'res_2', 'res_3', 'res_mean', 'station', 'total_var', 'undefs', 'vector', 'year']


In [22]:
#extract longitude and latitude for each station
feature='coeff_1'
sqlContext.registerDataFrameAsTable(df,'weather')
# Features=', '.join(['station', 'year','coeff_1', 'coeff_2', 'coeff_3', 'elevation', 'latitude', 'longitude',\
#          'res_1', 'res_2', 'res_3', 'res_mean', 'year'])
Features='station, year, coeff_2'
Query="SELECT %s FROM weather"%Features
print(Query)
pdf = sqlContext.sql(Query).toPandas()
pdf.head()

SELECT station, year, coeff_2 FROM weather


,station,year,coeff_2
0,USC00320941,1999.0,460.527436
1,USC00320941,1956.0,273.339803
2,CA005010191,2009.0,1343.217904
3,CA005010640,2011.0,96.440659
4,CA005010640,2006.0,646.845617


In [23]:
year_station_table=pdf.pivot(index='year', columns='station', values='coeff_2')
year_station_table.head(10)

station,CA004011250,CA004012485,CA004016521,CA004018960,CA005010191,CA005010480,CA005010485,CA005010640,CA005010QFQ,CA005012545,...,USC00327824,USC00328047,USC00328627,USC00328792,USC00328913,USC00328990,USC00329333,USC00329445,USW00024013,USW00094011
year,,,,,,,,,,,,,,,,,,,,,
1939.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1941.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1946.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1948.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1949.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,247.347400,-96.348883,NaN,NaN,-1350.401172,NaN,NaN,-890.617001,NaN
1950.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,53.773628,89.178716,NaN,-1170.720002,-755.968705,NaN,-77.704247,-332.428507,NaN
1951.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1766.926874,814.875445,NaN,NaN,216.816626,NaN,243.004439,244.855465,NaN
1952.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,322.499850,-70.139288,185.628256,-981.138987,-89.682004,-419.241028,NaN,NaN,-234.849054,NaN
1953.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,648.931623,NaN,NaN,245.451276,236.647033,NaN,NaN,246.037119,NaN,NaN


### Estimating the effect of the year vs the effect of the station

To estimate the effect of time vs. location on the first eigenvector coefficient we
compute:

* The average row: `mean-by-station`
* The average column: `mean-by-year`

We then compute the RMS before and after subtracting either  the row or the column vector.

In [24]:
def RMS(Mat):
    return (np.nanmean(Mat**2))

mean_by_year=np.nanmean(year_station_table,axis=1)
mean_by_station=np.nanmean(year_station_table,axis=0)
tbl_minus_year = (year_station_table.transpose()-mean_by_year).transpose()
tbl_minus_station = year_station_table-mean_by_station

a = RMS(year_station_table)
b = RMS(tbl_minus_station)
c = RMS(tbl_minus_year)

print 'total MS                   = ',a
print 'MS removing mean-by-station= ',b,', fraction explained= ', (a-b)/a*100
print 'MS removing mean-by-year   = ',c,', fraction explained= ', (a-c)/a*100

total MS                   =  417468.577365
MS removing mean-by-station=  381541.428529 , fraction explained=  8.60595282707
MS removing mean-by-year   =  151462.166332 , fraction explained=  63.7189061538


In [21]:
T=year_station_table
print 'initial RMS=',RMS(T)
for i in range(5):
    mean_by_year=np.nanmean(T,axis=1)
    T=(T.transpose()-mean_by_year).transpose()
    print i,'after removing mean by year    =',RMS(T)
    mean_by_station=np.nanmean(T,axis=0)
    T=T-mean_by_station
    print i,'after removing mean by stations=',RMS(T)

initial RMS= 646.118083143
0 after removing mean by year    = 389.181405429
0 after removing mean by stations= 364.382933589
1 after removing mean by year    = 363.235446874
1 after removing mean by stations= 363.111681619
2 after removing mean by year    = 363.090825353
2 after removing mean by stations= 363.085841244
3 after removing mean by year    = 363.084162625
3 after removing mean by stations= 363.083418988
4 after removing mean by year    = 363.083033867
4 after removing mean by stations= 363.082819911


In [1]:
sc.stop()

NameError: name 'sc' is not defined